In [ ]:
from dotenv import load_dotenv
import os
import logging
import time
import pandas as pd
from datetime import datetime
from binance import Client, exceptions
from prometheus_client import start_http_server, Gauge, Counter, Histogram
from requests.exceptions import ConnectionError, Timeout
from src.tradebot_setup import (
    initialize_metrics,
    update_metrics_on_buy,
    update_metrics_on_sell,
    buy_condition,
    sell_condition,
    process_buy,
    process_sell,
    safe_float_conversion,
    calculate_standard_deviation,
    calculate_percentage,
    update_trade_history,
    get_current_balance,
    get_lot_size,
    read_trade_history,
    check_last_transaction,
    check_buy_conditions
)

# Load environment variables from .env file
load_dotenv()

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

# Start Prometheus HTTP server on a separate thread
start_http_server(8000)

# Get API keys from environment variables
api_key = os.getenv('BINANCE_API_KEY')
api_secret = os.getenv('BINANCE_API_SECRET')

if not api_key or not api_secret:
    logger.error("API Key or API Secret not found. Check the .env file.")
    exit(1)

client = Client(api_key, api_secret, requests_params={'timeout': 20})

symbol = "BTCUSDT"
quantity = 0.0011  # Fixed amount of BTC to be transacted
interval = '1h'  # Candle time interval
setup = "9.1"  # Trading setup identifier
stopgain_percentage = 2  # Stopgain percentage
ema_span = 9  # EMA period

# Initialize metrics
metrics = initialize_metrics()
loop_counter_metric = Counter('loop_counter', 'Counts the number of loops', ['currency'])
current_price_metric = Gauge('current_price', 'Current price of the asset', ['currency'])
current_high_price_metric = Gauge('current_high_price', 'Current high price of the asset', ['currency'])
current_low_price_metric = Gauge('current_low_price', 'Current low price of the asset', ['currency'])
current_volume_metric = Gauge('current_volume', 'Current trading volume', ['currency'])
price_standard_deviation_metric = Gauge('price_standard_deviation', 'Standard deviation of closing prices', ['currency'])
sell_duration_metric = Histogram('sell_duration_seconds', 'Duration of sell transactions in seconds', ['currency'])
buy_duration_metric = Histogram('buy_duration_seconds', 'Duration of buy transactions in seconds', ['currency'])

# Define global variables
total_trade_duration = 0
total_trades = 0

def main_loop():
    global total_trade_duration, total_trades
    is_in_trade_logged = False
    is_not_in_trade_logged = False
    position_maintained = False

    while True:
        try:
            is_buy, trade_history = check_last_transaction(client, symbol)
            loop_counter_metric.labels(symbol).inc()

            if is_buy and not is_in_trade_logged:
                logger.info("Bot v2 started - Sell loop.")
                is_in_trade_logged = True
                is_not_in_trade_logged = False

            if not is_buy and not is_not_in_trade_logged:
                logger.info("Bot v2 started - Buy loop.")
                is_not_in_trade_logged = True

            while is_buy:
                if not position_maintained:
                    logger.info("Sell loop - Checking sell conditions.")
                    position_maintained = True
                ticker = float(client.get_symbol_ticker(symbol=symbol)['price'])

                if not trade_history.empty:
                    stoploss = trade_history['stoploss'].iloc[-1]
                    stopgain = trade_history['stopgain'].iloc[-1]

                    if ticker <= stoploss or ticker >= stopgain:
                        start_time = time.time()
                        balance_btc = get_current_balance(client, 'BTC')
                        lot_size = get_lot_size(client, symbol)
                        if balance_btc > 0 and lot_size:
                            quantity_to_sell = (balance_btc // lot_size) * lot_size
                            if quantity_to_sell > 0:
                                quantity_to_sell = round(quantity_to_sell, 8)
                                order = client.order_market_sell(symbol=symbol, quantity=quantity_to_sell)
                                trade_duration = time.time() - start_time
                                sell_duration_metric.labels(symbol).observe(trade_duration)
                                total_trade_duration += trade_duration
                                is_buy = False
                                logger.info("Sale executed.")
                                update_trade_history(trade_history, ticker, symbol)
                                update_metrics_on_sell(metrics, ticker, symbol)
                                position_maintained = False
                            else:
                                logger.info("Adjusted sell quantity is less than the lot size.")
                                is_buy = False
                                position_maintained = False
                        else:
                            logger.info("Insufficient BTC balance for sale.")
                            is_buy = False
                            position_maintained = False
                    time.sleep(1)

            while not is_buy:
                if not position_maintained:
                    logger.info("Buy loop - Checking buy conditions.")
                    position_maintained = True
                klines = client.get_klines(symbol=symbol, interval=interval, limit=50)
                data = pd.DataFrame(klines, columns=['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])

                data['close'] = data['close'].apply(safe_float_conversion)
                data['low'] = data['low'].apply(safe_float_conversion)
                data['high'] = data['high'].apply(safe_float_conversion)
                data['volume'] = data['volume'].apply(safe_float_conversion)

                if data[['close', 'low', 'high', 'volume']].isnull().any().any():
                    logger.error("Corrupted data received from Binance API.")
                    continue

                condition_met, current_price, previous_high = check_buy_conditions(data, ema_span)
                current_price_metric.labels(symbol).set(current_price)
                current_high_price_metric.labels(symbol).set(data['high'].max())
                current_low_price_metric.labels(symbol).set(data['low'].min())
                current_volume_metric.labels(symbol).set(data['volume'].sum())
                price_standard_deviation_metric.labels(symbol).set(calculate_standard_deviation(data['close']))

                if condition_met:
                    start_time = time.time()
                    order = client.order_market_buy(symbol=symbol, quantity=quantity)
                    buy_duration_metric.labels(symbol).observe(time.time() - start_time)
                    stoploss = data['low'].iloc[-2]
                    stopgain = current_price * (1 + stopgain_percentage / 100)
                    potential_loss = calculate_percentage(current_price, stoploss)
                    potential_gain = calculate_percentage(current_price, stopgain)
                    logger.info(f"Bought - Potential loss: {potential_loss:.2f}%, Potential gain: {potential_gain:.2f}%")
                    is_buy = True
                    new_row = pd.DataFrame({
                        'horario': [datetime.now()],
                        'moeda': [symbol],
                        'valor_compra': [current_price],
                        'valor_venda': [None],
                        'quantidade_moeda': [quantity],
                        'max_referencia': [previous_high],
                        'min_referencia': [data['low'].iloc[-2]],
                        'stoploss': [stoploss],
                        'stopgain': [stopgain],
                        'potential_loss': [potential_loss],
                        'potential_gain': [potential_gain],
                        'timeframe': [interval],
                        'setup': [setup],
                        'outcome': [None]
                    })
                    trade_history = pd.concat([trade_history, new_row], ignore_index=True)
                    trade_history.to_csv('data/trade_history.csv', index=False)
                    update_metrics_on_buy(metrics, current_price, stoploss, stopgain, potential_loss, potential_gain, symbol)
                    is_not_in_trade_logged = False
                    is_in_trade_logged = False
                time.sleep(1)

        except exceptions.BinanceAPIException as e:
            logger.error(f"Binance API error: {e}")
            time.sleep(25)
        except exceptions.BinanceOrderException as e:
            logger.error(f"Error creating order on Binance: {e}")
            time.sleep(25)
        except (ConnectionError, Timeout) as e:
            logger.error(f"Connection error: {e}")
            time.sleep(25)
        except Exception as e:
            logger.error(f"Unexpected error: {e}")
            time.sleep(25)

if __name__ == "__main__":
    main_loop()
